# LangChain

In [1]:
import os
import json
import pymongo
from typing import List
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import AzureCosmosDBVectorSearch
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.schema.document import Document
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import Tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_core.messages import SystemMessage

In [2]:
# Load settings for the notebook
load_dotenv()
CONNECTION_STRING = os.environ.get("DB_CONNECTION_STRING")
EMBEDDINGS_DEPLOYMENT_NAME = "text-embedding-3-small"
COMPLETIONS_DEPLOYMENT_NAME = "gpt-4"
AOAI_ENDPOINT = os.environ.get("AOAI_ENDPOINT")
AOAI_KEY = os.environ.get("AOAI_KEY")
AOAI_API_VERSION = "2023-09-01-preview"

In [8]:
# Establish Azure OpenAI connectivity
llm = AzureChatOpenAI(            
        temperature = 0,
        openai_api_version = AOAI_API_VERSION,
        azure_endpoint = AOAI_ENDPOINT,
        openai_api_key = AOAI_KEY,         
        azure_deployment = "completions"
        # azure_deployment = COMPLETIONS_DEPLOYMENT_NAME
)
embedding_model = AzureOpenAIEmbeddings(
    openai_api_version = AOAI_API_VERSION,
    azure_endpoint = AOAI_ENDPOINT,
    openai_api_key = AOAI_KEY,   
    azure_deployment = "embeddings",
    # azure_deployment = EMBEDDINGS_DEPLOYMENT_NAME,
    chunk_size=10
)

## Vector search with LangChain

In the previous lab, the `pymongo` library was used to perform a vector search through a db command to find product documents that were most similar to the user's input. In this lab, you will use the `langchain` library to perform the same search. LangChain has a vector store class named **AzureCosmosDBVectorSearch**, a community contribution, that supports vector search in vCore-based Azure Cosmos DB for MongoDB.

When establishing the connection to the vector store (vCore-based Azure Cosmos DB for MongoDB), recall that in previous labs the products collection was populated and a contentVector field added that contains the vectorized embeddings of the document itself. Finally, a vector index was also created on the contentVector field to enable vector search. The vector index in each collection is named `VectorSearchIndex`.

The return value of a vector search in LangChain is a list of `Document` objects. The LangChain `Document` class contains two properties: `page_content`, that represents the textual content that is typically used to augment the prompt, and `metadata` that contains all other attributes of the document. In the cell below, we'll use the `_id` field as the page_content, and the rest of the fields are returned as metadata.

The next two cells initiate a connection to the vector store and performs a vector search. Notice how much more concise the code is compared to the previous lab with the addition of LangChain.

In [9]:
# Reference the existing vector store
vector_store = AzureCosmosDBVectorSearch.from_connection_string(
    connection_string = CONNECTION_STRING,
    namespace = "cosmic_works.products",
    embedding = embedding_model,
    index_name = "VectorSearchIndex",    
    embedding_key = "contentVector",
    text_key = "_id"
)

In [11]:
query = "What yellow products are there?"
vector_store.similarity_search(query, k=3)

UnprocessableEntityError: Error code: 422 - {'detail': [{'type': 'string_type', 'loc': ['body', 'input', 'str'], 'msg': 'Input should be a valid string', 'input': [[3923, 2579, 3956, 527, 1070, 30]]}, {'type': 'string_type', 'loc': ['body', 'input', 'list[str]', 0], 'msg': 'Input should be a valid string', 'input': [3923, 2579, 3956, 527, 1070, 30]}]}

## RAG with LangChain

In this section, we'll implement the RAG pattern using LangChain. In LangChain, a **retriever** is used to augment the prompt with contextual data. In this case, the already established vector store will be used as the retriever. By default, the prompt is augmented with the `page_content` field of the retrieved document that customarily contains the text content of the embedded vector. In our case, the document itself serves as the textual content, so we'll have to do some pre-processing to format the text of the product list that is expected in our system prompt (JSON string) - see the **format_documents** function below for this implementation.

We'll also define a reusable RAG [chain](https://python.langchain.com/docs/modules/chains/) to control the flow and behavior of the call into the LLM. This chain is defined using the LCEL syntax (LangChain Expression Language).

In [ ]:
# A system prompt describes the responsibilities, instructions, and persona of the AI.
# Note the addition of the templated variable/placeholder for the list of products and the incoming question.
system_prompt = """
You are a helpful, fun and friendly sales assistant for Cosmic Works, a bicycle and bicycle accessories store. 
Your name is Cosmo.
You are designed to answer questions about the products that Cosmic Works sells.

Only answer questions related to the information provided in the list of products below that are represented
in JSON format.

If you are asked a question that is not in the list, respond with "I don't know."

Only answer questions related to Cosmic Works products, customers, and sales orders.

If a question is not related to Cosmic Works products, customers, or sales orders,
respond with "I only answer questions about Cosmic Works"

List of products:
{products}

Question:
{question}
"""

In [ ]:
# remember that each Document contains a page_content property
# that is populated with the _id field of the document
# all other document fields are located in the metadata property
def format_docs(docs:List[Document]) -> str:
        """
        Prepares the product list for the system prompt.
        """
        str_docs = []
        for doc in docs:
                # Build the product document without the contentVector
                doc_dict = {"_id": doc.page_content}
                doc_dict.update(doc.metadata)
                if "contentVector" in doc_dict:  
                        del doc_dict["contentVector"]
                str_docs.append(json.dumps(doc_dict, default=str))                  
        # Return a single string containing each product JSON representation
        # separated by two newlines
        return "\n\n".join(str_docs)

In [ ]:
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create the prompt template from the system_prompt text
llm_prompt = PromptTemplate.from_template(system_prompt)

rag_chain = (
    # populate the tokens/placeholders in the llm_prompt 
    # products takes the results of the vector store and formats the documents
    # question is a passthrough that takes the incoming question
    { "products": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    # pass the populated prompt to the language model
    | llm
    # return the string ouptut from the language model
    | StrOutputParser()
)

In [ ]:
question = "What are the names and skus of yellow products? Output the answer as a bulleted list."
response = rag_chain.invoke(question)
print(response)

## LangChain Agent

### Create retrievers

A separate retriever is required for each vector index. The following cell creates a VectorStoreRetriever for the products, customers, and sales collections and associated vector index.

In [ ]:
def create_cosmic_works_vector_store_retriever(collection_name: str, top_k: int = 3):
    vector_store =  AzureCosmosDBVectorSearch.from_connection_string(
        connection_string = CONNECTION_STRING,
        namespace = f"cosmic_works.{collection_name}",
        embedding = embedding_model,
        index_name = "VectorSearchIndex",    
        embedding_key = "contentVector",
        text_key = "_id"
    )
    return vector_store.as_retriever(search_kwargs={"k": top_k})


products_retriever = create_cosmic_works_vector_store_retriever("products")
customers_retriever = create_cosmic_works_vector_store_retriever("customers")
sales_retriever = create_cosmic_works_vector_store_retriever("sales")

 ### Create Agent Tools
 
 LangChain does have a built-in [`create_retriever_tool`](https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents#retriever-tool) that wraps a vector store retriever, however, because we are storing the embeddings in the `contentVector` field of the document, we must do some pre-processing of the retrieved documents to remove this field so that we don't needlessly expend the model's token quota. 
 
 Instead, we'll create a RAG chain as our tool implementation that does the pre-processing through the `format_docs` function we defined above to return each document in its JSON representation.

In [ ]:
# Create tools that will use vector search in vCore-based Azure Cosmos DB for MongoDB collections

# create a chain on the retriever to format the documents as JSON
products_retriever_chain = products_retriever | format_docs
customers_retriever_chain = customers_retriever | format_docs
sales_retriever_chain = sales_retriever | format_docs

tools = [
    Tool(
        name = "vector_search_products", 
        func = products_retriever_chain.invoke,
        description = "Searches Cosmic Works product information for similar products based on the question. Returns the product information in JSON format."
    ),
    Tool(
        name = "vector_search_customers", 
        func = customers_retriever_chain.invoke,
        description = "Searches Cosmic Works customer information and retrieves similar customers based on the question. Returns the customer information in JSON format."
    ),
    Tool(
        name = "vector_search_sales", 
        func = sales_retriever_chain.invoke,
        description = "Searches Cosmic Works customer sales information and retrieves sales order details based on the question. Returns the sales order information in JSON format."
    )
]

### Tools part 2

Certain properties do not have semantic meaning (such as the GUID _id fields) and attempting to use vector search on these fields will not yield meaningful results. We need a tool to retrieve specific documents based on popular searches criteria.

The following tool definitions is not an exhaustive list of what may be needed but serves as an example to provide concrete lookups of a document in the Cosmic Works database.

In [ ]:
db = pymongo.MongoClient(CONNECTION_STRING).cosmic_works

def get_product_by_id(product_id: str) -> str:
    """
    Retrieves a product by its ID.    
    """
    doc = db.products.find_one({"_id": product_id})    
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc)

def get_product_by_sku(sku: str) -> str:
    """
    Retrieves a product by its sku.
    """
    doc = db.products.find_one({"sku": sku})
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc, default=str)

def get_sales_by_id(sales_id: str) -> str:
    """
    Retrieves a sales order by its ID.
    """
    doc = db.sales.find_one({"_id": sales_id})
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc, default=str)    

from langchain.tools import StructuredTool

tools.extend([
    StructuredTool.from_function(get_product_by_id),
    StructuredTool.from_function(get_product_by_sku),
    StructuredTool.from_function(get_sales_by_id)
])

### Create the agent

The [`create_conversational_retrieval_agent`](https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents#agent-constructor) is a built-in agent that includes conversational history as well uses the [OpenAIFunctionsAgent](https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent#using-openaifunctionsagent) as its underlying implementation.

Remember that an agent leverages the LLM to assess the incoming request with the current context to decide what steps or actions need to be executed and in what order. LangChain agents can leverage tools. A tool can be an integration into an external system, custom code, or even another chain.

In [ ]:
system_message = SystemMessage(
    content = """
        You are a helpful, fun and friendly sales assistant for Cosmic Works, a bicycle and bicycle accessories store.

        Your name is Cosmo.

        You are designed to answer questions about the products that Cosmic Works sells, the customers that buy them, and the sales orders that are placed by customers.

        If you don't know the answer to a question, respond with "I don't know."
        
        Only answer questions related to Cosmic Works products, customers, and sales orders.
        
        If a question is not related to Cosmic Works products, customers, or sales orders,
        respond with "I only answer questions about Cosmic Works"
    """    
)
agent_executor = create_conversational_retrieval_agent(llm, tools, system_message = system_message, verbose=True)

In [ ]:
result = agent_executor({"input": "What products do you have that are yellow?"})
print("***********************************************************")
print(result['output'])

In [ ]:
result = agent_executor({"input": "What products were purchased for sales order '06FE91D2-B350-471A-AD29-906BF4EB97C4' ?"})
print("***********************************************************")
print(result['output'])

In [ ]:
result = agent_executor({"input": "What was the sales order total for sales order '93436616-4C8A-407D-9FDA-908707EFA2C5' ?"})
print("***********************************************************")
print(result['output'])

In [ ]:
result = agent_executor({"input": "What was the price of the product with sku `FR-R92B-58` ?"})
print("***********************************************************")
print(result['output'])